In [1]:
path_datos = "gs://bucket-tfm-llc/datos"

In [2]:
# print(os.listdir("/content/drive/MyDrive/TFM/datos/"))

# Librerias

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Lectura tablas

In [4]:
df_amzn = spark.read.parquet(f"{path_datos}/datos_financieros_amzn_raw")

In [5]:
df_amzn.show()

+-------------------+-------+-------+------+------+-----------+
|               date|   open|   high|   low| close|     volume|
+-------------------+-------+-------+------+------+-----------+
|2025-01-31 00:00:00|  236.5| 240.29|236.41|237.68|3.6162377E7|
|2025-01-30 00:00:00| 237.14| 237.95|232.22|234.64|3.2020728E7|
|2025-01-29 00:00:00|239.015| 240.39|236.15|237.07|2.6091716E7|
|2025-01-28 00:00:00| 234.29| 241.77|233.98|238.15|4.1587188E7|
|2025-01-27 00:00:00| 226.21| 235.61|225.86|235.42|4.9428332E7|
|2025-01-24 00:00:00|  234.5|  236.4|232.93|234.85|2.5890738E7|
|2025-01-23 00:00:00|  234.1| 235.52|231.51|235.42|2.6404364E7|
|2025-01-22 00:00:00| 232.02| 235.44|231.19|235.01|4.1448217E7|
|2025-01-21 00:00:00|  228.9| 231.78|226.94|230.71|3.9951456E7|
|2025-01-17 00:00:00| 225.84| 226.51|223.08|225.94|4.2370123E7|
|2025-01-16 00:00:00| 224.42| 224.65|220.31|220.66|2.4757276E7|
|2025-01-15 00:00:00| 222.83| 223.57|220.75|223.35|3.1291257E7|
|2025-01-14 00:00:00| 220.44| 221.82| 21

In [6]:
df_amzn.summary().show()

+-------+-----------------+-----------------+-----------------+-----------------+--------------------+
|summary|             open|             high|              low|            close|              volume|
+-------+-----------------+-----------------+-----------------+-----------------+--------------------+
|  count|             6309|             6309|             6309|             6309|                6309|
|   mean|593.3223745760007|600.0108554446048|585.8828748930082|593.0824456966238|1.1306734247424314E7|
| stddev|927.8425149015573| 938.137159963499|916.1059603175175|927.0595077718248|1.7253855835222587E7|
|    min|             5.91|              6.1|             5.51|             5.97|            881337.0|
|    25%|            47.22|             48.2|            46.06|            47.22|           3648791.0|
|    50%|            165.5|           168.47|         162.8701|            165.8|           5625700.0|
|    75%|           637.14|           645.93|           627.35|          

# Tratamiento - Feature engineering

## Target

In [7]:
# Cálculo de la variable target
# Para evitar ruido, miramos un horizonte de 5 días y que la subida sea mayor al 2%:
w_temporal = Window.orderBy("date")
pct_subida = 0.02

# df_amzn_target = df_amzn.withColumn("target", F.when(F.lead("close").over(w_temporal) > F.col("close"), F.lit(1)).otherwise(F.lit(0)))
df_amzn_target = df_amzn.withColumn("target", F.when(F.lead("close", 5).over(w_temporal) > F.col("close") * (1 + pct_subida), F.lit(1)).otherwise(F.lit(0)))

## Tendencia

In [8]:
# Cálculo de ratios de cambio en distintas ventanas temporales:
df_amzn_ratios = df_amzn_target.\
      withColumn("roc_5d", (F.col("close") - F.lag("close",5).over(w_temporal)) / F.lag("close",5).over(w_temporal)).\
      withColumn("roc_10d", (F.col("close") - F.lag("close",10).over(w_temporal)) / F.lag("close",10).over(w_temporal)).\
      withColumn("roc_20d", (F.col("close") - F.lag("close",20).over(w_temporal)) / F.lag("close",20).over(w_temporal)).\
      withColumn("roc_50d", (F.col("close") - F.lag("close",50).over(w_temporal)) / F.lag("close",50).over(w_temporal)).\
      withColumn("roc_100d", (F.col("close") - F.lag("close",100).over(w_temporal)) / F.lag("close",100).over(w_temporal)).\
      withColumn("roc_200d", (F.col("close") - F.lag("close",200).over(w_temporal)) / F.lag("close",200).over(w_temporal))


In [9]:
# Cálculo de medias móviles (MA), momentum (diferencia con las MA) y Bollinger Bands:
df_amzn_ma = df_amzn_ratios.\
      withColumn("ma_5d", F.avg("close").over(Window.orderBy("date").rowsBetween(-5, 0))).\
      withColumn("ma_10d", F.avg("close").over(Window.orderBy("date").rowsBetween(-9, 0))).\
      withColumn("ma_20d", F.avg("close").over(Window.orderBy("date").rowsBetween(-19, 0))).\
      withColumn("ma_50d", F.avg("close").over(Window.orderBy("date").rowsBetween(-49, 0))).\
      withColumn("ma_100d", F.avg("close").over(Window.orderBy("date").rowsBetween(-99, 0))).\
      withColumn("ma_200d", F.avg("close").over(Window.orderBy("date").rowsBetween(-199, 0))).\
      withColumn("std_20", F.stddev("close").over(Window.orderBy("date").rowsBetween(-19, 0))).\
      withColumn("boll_upper_20d", F.col("ma_20d") + 2 * F.col("std_20")).\
      withColumn("boll_lower_20d", F.col("ma_20d") - 2 * F.col("std_20")).\
      drop("std_20")

In [10]:
# Cálculo del z-score de cierre, indica cómo de "extremo" es el precio
df_amzn_zscore = df_amzn_ma.\
      withColumn("std_5d", F.stddev("close").over(Window.orderBy("date").rowsBetween(-4, 0))).\
      withColumn("std_10d", F.stddev("close").over(Window.orderBy("date").rowsBetween(-9, 0))).\
      withColumn("std_20d", F.stddev("close").over(Window.orderBy("date").rowsBetween(-19, 0))).\
      withColumn("std_50d", F.stddev("close").over(Window.orderBy("date").rowsBetween(-49, 0))).\
      withColumn("std_100d", F.stddev("close").over(Window.orderBy("date").rowsBetween(-99, 0))).\
      withColumn("std_200d", F.stddev("close").over(Window.orderBy("date").rowsBetween(-199, 0))).\
      withColumn("zscore_5d", F.when(F.col("std_5d")!=0, (F.col("close") - F.col("ma_5d")) / F.col("std_5d")).otherwise(None)).\
      withColumn("zscore_10d", F.when(F.col("std_10d")!=0, (F.col("close") - F.col("ma_10d")) / F.col("std_10d")).otherwise(None)).\
      withColumn("zscore_20d", F.when(F.col("std_20d")!=0, (F.col("close") - F.col("ma_20d")) / F.col("std_20d")).otherwise(None)).\
      withColumn("zscore_50d", F.when(F.col("std_50d")!=0, (F.col("close") - F.col("ma_50d")) / F.col("std_50d")).otherwise(None)).\
      withColumn("zscore_100d", F.when(F.col("std_100d")!=0, (F.col("close") - F.col("ma_100d")) / F.col("std_100d")).otherwise(None)).\
      withColumn("zscore_200d", F.when(F.col("std_200d")!=0, (F.col("close") - F.col("ma_200d")) / F.col("std_200d")).otherwise(None)).\
      drop("std_5d", "std_10d", "std_20d", "std_50d", "std_100d", "std_200d")

In [11]:
# Relaciones entre medias móviles:
# Diferencia entre las medias móviles (MA Crossovers) para incluir la relación entre estos indicadores
df_amzn_ma_rel = df_amzn_zscore.\
      withColumn("ma_10d_vs_20d", F.col("ma_20d") - F.col("ma_10d")).\
      withColumn("ma_20d_vs_50d", F.col("ma_50d") - F.col("ma_20d")).\
      withColumn("ma_50d_vs_100d", F.col("ma_100d") - F.col("ma_50d")).\
      withColumn("ma_100d_vs_200d", F.col("ma_200d") - F.col("ma_100d")).\
      withColumn("cross_ma_10d_vs_20d", F.when(F.col("ma_10d") > F.col("ma_20d"), 1).otherwise(0)).\
      withColumn("cross_ma_20d_vs_50d", F.when(F.col("ma_20d") > F.col("ma_50d"), 1).otherwise(0)).\
      withColumn("cross_ma_50d_vs_100d", F.when(F.col("ma_50d") > F.col("ma_100d"), 1).otherwise(0)).\
      withColumn("cross_ma_100d_vs_200d", F.when(F.col("ma_100d") > F.col("ma_200d"), 1).otherwise(0))

## Volatilidad

In [12]:
# Cálculo de la volatilidad:
w_volatilidad = Window.orderBy("date").rowsBetween(-5, 0)
df_amzn_vol = df_amzn_ma_rel.\
      withColumn("volatility_5d", F.stddev("close").over(Window.orderBy("date").rowsBetween(-4, 0))).\
      withColumn("volatility_10d", F.stddev("close").over(Window.orderBy("date").rowsBetween(-9, 0))).\
      withColumn("volatility_20d", F.stddev("close").over(Window.orderBy("date").rowsBetween(-19, 0)))

In [13]:
# Volatilidad diaria (ATR- Average True Range):
df_amzn_atr = df_amzn_vol.\
      withColumn("high_low", F.col("high") - F.col("low")).\
      withColumn("high_prev_close", F.abs(F.col("high") - F.lag("close",1).over(w_temporal))).\
      withColumn("low_prev_close", F.abs(F.col("low") - F.lag("close",1).over(w_temporal))).\
      withColumn("true_range", F.greatest("high_low", "high_prev_close", "low_prev_close")).\
      withColumn("atr_7d", F.avg("true_range").over(Window.orderBy("date").rowsBetween(-6, 0))).\
      withColumn("atr_14d", F.avg("true_range").over(Window.orderBy("date").rowsBetween(-13, 0))).\
      withColumn("atr_21d", F.avg("true_range").over(Window.orderBy("date").rowsBetween(-20, 0))).\
      drop("high_low", "high_prev_close", "low_prev_close", "true_range")

In [14]:
# Diferencia entre open y close anterior (movimiento nocturno)
df_amzn_gap = df_amzn_atr.\
      withColumn("gap_open", F.col("open") - F.lag("close", 1).over(w_temporal)).\
      withColumn("gap_pct", (F.col("gap_open"))/ F.lag("close", 1).over(w_temporal))

## Volumen

In [15]:
# Volumen:
df_amzn_vol = df_amzn_gap.\
    withColumn("volume_avg_5d", F.avg("volume").over(Window.orderBy("date").rowsBetween(-4, 0))).\
    withColumn("volume_avg_10d", F.avg("volume").over(Window.orderBy("date").rowsBetween(-9, 0))).\
    withColumn("volume_avg_20d", F.avg("volume").over(Window.orderBy("date").rowsBetween(-19, 0))).\
    withColumn("volume_ratio_5d", F.col("volume") / F.col("volume_avg_5d")).\
    withColumn("volume_ratio_10d", F.col("volume") / F.col("volume_avg_10d")).\
    withColumn("volume_ratio_20d", F.col("volume") / F.col("volume_avg_20d"))

In [16]:
# Money flow:
df_amzn_mf = df_amzn_vol.withColumn("money_flow", (F.col("close")-F.col("low")) - (F.col("high")-F.col("close")) * F.col("volume"))

## Momentum y osciladores

In [17]:
# RSI (Relative Strength Index - 7, 14 y 21 días):
df_amzn_rsi = df_amzn_mf.withColumn("delta", F.col("close") - F.lag("close", 1).over(w_temporal)).\
    withColumn("gain", F.when(F.col("delta") > 0, F.col("delta")).otherwise(0)).\
    withColumn("loss", F.when(F.col("delta") < 0, -F.col("delta")).otherwise(0)).\
    withColumn("avg_gain_7d", F.avg("gain").over(Window.orderBy("date").rowsBetween(-6, 0))).\
    withColumn("avg_loss_7d", F.avg("loss").over(Window.orderBy("date").rowsBetween(-6, 0))).\
    withColumn("avg_gain_14d", F.avg("gain").over(Window.orderBy("date").rowsBetween(-13, 0))).\
    withColumn("avg_loss_14d", F.avg("loss").over(Window.orderBy("date").rowsBetween(-13, 0))).\
    withColumn("avg_gain_21d", F.avg("gain").over(Window.orderBy("date").rowsBetween(-20, 0))).\
    withColumn("avg_loss_21d", F.avg("loss").over(Window.orderBy("date").rowsBetween(-20, 0))).\
    withColumn("rs_7d", F.col("avg_gain_7d") / (F.col("avg_loss_7d")+ 1e-10)).\
    withColumn("rs_14d", F.col("avg_gain_14d") / (F.col("avg_loss_14d")+ 1e-10)).\
    withColumn("rs_21d", F.col("avg_gain_21d") / (F.col("avg_loss_21d")+ 1e-10)).\
    withColumn("rsi_7d", 100 - (100 / (1 + F.col("rs_7d")))).\
    withColumn("rsi_14d", 100 - (100 / (1 + F.col("rs_14d")))).\
    withColumn("rsi_21d", 100 - (100 / (1 + F.col("rs_21d")))).\
    select(*df_amzn_mf.columns + ["rsi_7d","rsi_14d","rsi_21d"])


In [18]:
# Cambios en el RSI para identificar momentos de aceleración o reversión
df_amzn_rsi_rel = df_amzn_rsi.\
    withColumn("change_rsi_7d", F.col("rsi_7d") - F.lag("rsi_7d",1).over(w_temporal)).\
    withColumn("change_rsi_14d", F.col("rsi_14d") - F.lag("rsi_14d",1).over(w_temporal)).\
    withColumn("change_rsi_21d", F.col("rsi_21d") - F.lag("rsi_21d",1).over(w_temporal))

In [19]:
# Calculamos el indicador de Williams %R para distintas ventanas temporales
# También calculamos el oscilador estocástico %K (similar a Williams %R pero más suave)
df_amzn_ind = df_amzn_rsi_rel.\
      withColumn("max_high_7d", F.max("high").over(Window.orderBy("date").rowsBetween(-6, 0))).\
      withColumn("max_high_14d", F.max("high").over(Window.orderBy("date").rowsBetween(-13, 0))).\
      withColumn("max_high_21d", F.max("high").over(Window.orderBy("date").rowsBetween(-20, 0))).\
      withColumn("min_low_7d", F.min("low").over(Window.orderBy("date").rowsBetween(-6, 0))).\
      withColumn("min_low_14d", F.min("low").over(Window.orderBy("date").rowsBetween(-13, 0))).\
      withColumn("min_low_21d", F.min("low").over(Window.orderBy("date").rowsBetween(-20, 0))).\
    withColumn("williams_r_7d", ((F.col("max_high_7d") - F.col("close")) / (F.col("max_high_7d") - F.col("min_low_7d"))) * -100).\
    withColumn("williams_r_14d", ((F.col("max_high_14d") - F.col("close")) / (F.col("max_high_14d") - F.col("min_low_14d"))) * -100).\
    withColumn("williams_r_21d", ((F.col("max_high_21d") - F.col("close")) / (F.col("max_high_21d") - F.col("min_low_21d"))) * -100).\
    withColumn("eo_k_7d", ((F.col("close") - F.col("min_low_7d")) / (F.col("max_high_7d") - F.col("min_low_7d"))) * 100).\
    withColumn("eo_k_14d", ((F.col("close") - F.col("min_low_14d")) / (F.col("max_high_14d") - F.col("min_low_14d"))) * 100).\
    withColumn("eo_k_21d", ((F.col("close") - F.col("min_low_21d")) / (F.col("max_high_21d") - F.col("min_low_21d"))) * 100).\
    select(*df_amzn_rsi_rel.columns + ["williams_r_7d", "williams_r_14d", "williams_r_21d", "eo_k_7d", "eo_k_14d", "eo_k_21d"])

## Temporal

In [20]:
# Añadimos algunas variables temporales
df_amzn_tmp = df_amzn_ind.\
      withColumn("day_of_week", F.dayofweek("date")).\
      withColumn("day_of_month", F.dayofmonth("date")).\
      withColumn("month", F.month("date")).\
      withColumn("week_of_year", F.weekofyear("date"))

# Output

In [21]:
len(df_amzn_tmp.columns)

66

In [22]:
# Guardamos los datos tras realizar el feature engineering
df_amzn_tmp.write.mode("overwrite").format("parquet").save(f"{path_datos}/datos_financieros_amzn_trat")

25/06/07 10:18:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/07 10:18:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/07 10:18:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/07 10:18:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/07 10:18:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/07 10:18:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/07 1

In [24]:
df_amzn_tmp.count()

6309